# Testando Detectores

In [546]:
import cv2
import numpy as np
import pandas as pd
import itertools

## Funções

In [1020]:
def detectHarrisKeypoints(image, threshold=0.01, blockSize=2, ksize=3, k=0.04):
    # Reading the image and converting the image to B/W 
    gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY) 
    gray_image_f32 = np.float32(gray_image)

    # Applying the function 
    dst = cv2.cornerHarris(gray_image_f32, blockSize, ksize, k) 
  
    # dilate to mark the corners 
    dst = cv2.dilate(dst, None)
    
    ret, dst = cv2.threshold(dst,threshold*dst.max(),255,0)
    dst = np.uint8(dst)
    
    # find centroids
    ret, labels, stats, centroids = cv2.connectedComponentsWithStats(dst)

    # define the criteria to stop and refine the corners
    criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 100, 0.001)
    corners = cv2.cornerSubPix(gray_image_f32,np.float32(centroids),(5,5),(-1,-1),criteria)

    # # extract keypoints
    # points = np.argwhere(dst > threshold * dst.max())
    
    keypoints = [cv2.KeyPoint(float(x[0]), float(x[1]), 13) for x in corners]

    # draw keypoints
    # image[dst > threshold * dst.max()] = [0, 255, 0]
    kp_image = cv2.drawKeypoints(image, keypoints, None, color=(255, 0, 0), flags=0)

    return keypoints, kp_image


In [1021]:
def detectSIFTKeypoints(image, nfeatures=0, nOctaveLayers=3, contrastThreshold=0.04, edgeThreshold=10, sigma=1.6, enable_precise_upscale=False):
    # Reading the image and converting the image to B/W 
    gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY) 
  
    # Applying the function 
    sift = cv2.SIFT_create(nfeatures, nOctaveLayers, contrastThreshold, edgeThreshold, sigma, enable_precise_upscale) 
    kp, des = sift.detectAndCompute(gray_image, None)
    not_dup_kp = {pt.pt: pt for pt in kp}
    kp_ = list(not_dup_kp.values())
    # Applying the function 
    kp_image = cv2.drawKeypoints(image, kp_, None, color=(0, 255, 0), flags=0)

    return kp_, kp_image 

def detectSIFTKeypointsFilter(image, nfeatures=0, nOctaveLayers=3, contrastThreshold=0.04, edgeThreshold=10, sigma=1.6, enable_precise_upscale=False):
    # Reading the image and converting the image to B/W 
    gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY) 
    gray_image_f32 = np.float32(gray_image)
  
    # Applying the function 
    sift = cv2.SIFT_create(nfeatures, nOctaveLayers, contrastThreshold, edgeThreshold, sigma, enable_precise_upscale) 
    kp, des = sift.detectAndCompute(gray_image, None)
    not_dup_kp = {pt.pt: pt for pt in kp}
    kp_ = list(not_dup_kp.values())
    
    altura, largura = image.shape[:2]
    imagem_binaria = np.zeros((altura, largura), dtype=np.uint8)
    for kp in kp_:
        x, y = map(int, kp.pt)
        cv2.circle(imagem_binaria, (x, y), 5, 255, -1)

    # find centroids
    ret, labels, stats, centroids = cv2.connectedComponentsWithStats(imagem_binaria)

    # define the criteria to stop and refine the corners
    criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 100, 0.001)
    corners = cv2.cornerSubPix(gray_image_f32,np.float32(centroids),(5,5),(-1,-1),criteria)

    # # extract keypoints
    # points = np.argwhere(dst > threshold * dst.max())
    
    keypoints = [cv2.KeyPoint(float(x[0]), float(x[1]), 13) for x in corners]
    
    # Applying the function 
    kp_image = cv2.drawKeypoints(image, keypoints, None, color=(0, 255, 0), flags=0)

    return keypoints, kp_image 

In [1022]:
def detectStarKeypoints(image, max_size = 41, response_threshold = 30, line_threshold_projected = 10,
                        line_threshold_binarized = 8, suppress_nonmax_size = 5):
    # Reading the image and converting the image to B/W 
    gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY) 
  
    # Applying the function 
    star = cv2.xfeatures2d.StarDetector_create(maxSize= max_size, 
                                        responseThreshold = response_threshold,
                                        lineThresholdProjected = line_threshold_projected,
                                        lineThresholdBinarized = line_threshold_binarized,
                                        suppressNonmaxSize = suppress_nonmax_size)
    kp = star.detect(gray_image, None)

    # Applying the function 
    kp_image = cv2.drawKeypoints(image, kp, None, color=(0, 0, 255), flags=0) 

    return kp, kp_image 


def detectStarKeypointsFilter(image, max_size = 41, response_threshold = 30, line_threshold_projected = 10,
                        line_threshold_binarized = 8, suppress_nonmax_size = 5):
    # Reading the image and converting the image to B/W 
    gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    gray_image_f32 = np.float32(gray_image)
  
    # Applying the function 
    star = cv2.xfeatures2d.StarDetector_create(maxSize= max_size, 
                                        responseThreshold = response_threshold,
                                        lineThresholdProjected = line_threshold_projected,
                                        lineThresholdBinarized = line_threshold_binarized,
                                        suppressNonmaxSize = suppress_nonmax_size)
    kps = star.detect(gray_image, None)

    altura, largura = image.shape[:2]
    imagem_binaria = np.zeros((altura, largura), dtype=np.uint8)
    for kp in kps:
        x, y = map(int, kp.pt)
        cv2.circle(imagem_binaria, (x, y), 5, 255, -1)

    # find centroids
    ret, labels, stats, centroids = cv2.connectedComponentsWithStats(imagem_binaria)

    # define the criteria to stop and refine the corners
    criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 100, 0.001)
    corners = cv2.cornerSubPix(gray_image_f32,np.float32(centroids),(5,5),(-1,-1),criteria)

    # # extract keypoints
    # points = np.argwhere(dst > threshold * dst.max())
    
    keypoints = [cv2.KeyPoint(float(x[0]), float(x[1]), 13) for x in corners]

    # Applying the function 
    kp_image = cv2.drawKeypoints(image, keypoints, None, color=(0, 0, 255), flags=0) 

    return keypoints, kp_image 

## Testando parâmetros Harris

In [1023]:
# image = cv2.imread('../data/imgs/dsc07631.jpg')
# image = cv2.imread('../data/imgs/dsc02651.jpg')
image = cv2.imread('../data/imgs_teste_3/IMG-20240114-WA0043.jpg')

# image = cv2.cvtColor(image, cv2.COLOR_RGB2Lab)
# clahe = cv2.createCLAHE(clipLimit=1, tileGridSize=(50,50))
# image[:,:,0] = clahe.apply(image[:,:,0])
# image = cv2.cvtColor(image, cv2.COLOR_Lab2RGB)

image = cv2.resize(image, (640,480))

kp, kp_image = detectHarrisKeypoints(image, threshold=0.005, blockSize=6, ksize=3, k=0.03)
print(len(kp))
cv2.imshow('Harris', kp_image) 
cv2.waitKey() 

22


-1

## Testando parâmetros SIFT

In [1050]:
# image = cv2.imread('../data/imgs/dsc07631.jpg')
# image = cv2.imread('../data/imgs/dsc02651.jpg')
image = cv2.imread('../data/imgs_teste_3/IMG-20240114-WA0043.jpg')
image = cv2.resize(image, (640,480))
kp, kp_image = detectSIFTKeypointsFilter(image, 
                                   nfeatures=52,
                                   nOctaveLayers=1,
                                   contrastThreshold=0.02,
                                   edgeThreshold=5,
                                   sigma=1.6,
                                   enable_precise_upscale=True)
# print(len(set([p.pt for p in kp])))
print(len(kp))
cv2.imshow('SIFT', kp_image) 
cv2.waitKey() 

24


-1

## Testando parâmetros STAR

In [1025]:
# image = cv2.imread('../data/imgs/dsc07631.jpg')
# image = cv2.imread('../data/imgs/dsc02651.jpg')
image = cv2.imread('../data/imgs_teste_3/IMG-20240114-WA0043.jpg')
image = cv2.resize(image, (640,480))
kp, kp_image = detectStarKeypointsFilter(image, 
                                   max_size = 10, 
                                   response_threshold = 10, 
                                   line_threshold_projected = 5,
                                   line_threshold_binarized = 5, 
                                   suppress_nonmax_size = 1)
# print(len(set([p.pt for p in kp])))
print(len(kp))
cv2.imshow('Harris', kp_image) 
cv2.waitKey() 

21


-1

## Comparando os pontos dos detectores

### Funções

In [1026]:
from sklearn.metrics import pairwise_distances_argmin_min

def computeDistacesKeypoints(pts1, pts2, threshold=1):
    array_pts1 = np.asarray(pts1)
    array_pts2 = np.asarray(pts2)

    if array_pts1.shape[0] > 0 and array_pts2.shape[0] > 0:
        dists = pairwise_distances_argmin_min(array_pts1, array_pts2)
        matches_pts1 = [pts1[i] for i, (pt, dist) in enumerate(zip(dists[0], dists[1])) if dist <= threshold]
        matches_pts2 = [pts2[pt] for pt, dist in zip(dists[0], dists[1]) if dist <= threshold]
    else:
        matches_pts1 = []
        matches_pts2 = []
        
    return matches_pts1, matches_pts2

In [1027]:
def return_detector_func(algol):
    if algol == 'harris':
        func = detectHarrisKeypoints
    elif algol == 'sift_filter':
        func = detectSIFTKeypointsFilter
    elif algol == 'sift':
        func = detectSIFTKeypoints
    elif algol == 'star_filter':
        func = detectStarKeypointsFilter
    else:
        func = detectStarKeypoints
    
    return func


def compare_detectors_keypoints(image, algol_1, algol_2, params_algol_1, params_algol_2, threshold):
    detector_1 = return_detector_func(algol_1)
    detector_2 = return_detector_func(algol_2)

    kp1, _ = detector_1(image, **params_algol_1)
    kp2, _ = detector_2(image, **params_algol_2)

    kp1 = [(kp.pt[0], kp.pt[1]) for kp in kp1]
    kp2 = [(kp.pt[0], kp.pt[1]) for kp in kp2]

    matches_pts1, matches_pts2 = computeDistacesKeypoints(kp1, kp2, threshold=threshold)
    mismatches_pts1 = [pt for pt in kp1 if pt not in matches_pts1]
    mismatches_pts2 = [pt for pt in kp2 if pt not in matches_pts2]

    matches_kp1 = [cv2.KeyPoint(float(x[0]), float(x[1]), 13) for x in matches_pts1]
    matches_kp2 = [cv2.KeyPoint(float(x[0]), float(x[1]), 13) for x in matches_pts2]

    mismatches_kp1 = [cv2.KeyPoint(float(x[0]), float(x[1]), 13) for x in mismatches_pts1]
    mismatches_kp2 = [cv2.KeyPoint(float(x[0]), float(x[1]), 13) for x in mismatches_pts2]

    kp_image = cv2.drawKeypoints(image, matches_kp1, None, color=(0, 255, 0), flags=0)
    kp_image = cv2.drawKeypoints(kp_image, matches_kp2, None, color=(0, 255, 0), flags=0)
    kp_image = cv2.drawKeypoints(kp_image, mismatches_kp1, None, color=(255, 0, 0), flags=0)
    kp_image = cv2.drawKeypoints(kp_image, mismatches_kp2, None, color=(0, 0, 255), flags=0)

    return kp_image, matches_kp1, matches_kp2, kp1, kp2 

def return_params_combination(combination, params_harris, params_sift, params_star, pos):
    if combination == 'harris':
        params = {key: value[pos] for key, value in zip(params_harris.keys(), params_harris.values())}
    elif combination == 'sift' or combination == 'sift_filter':
        params = {key: value[pos] for key, value in zip(params_sift.keys(), params_sift.values())}
    else:
        params = {key: value[pos] for key, value in zip(params_star.keys(), params_star.values())}
    
    return params


def run_all_tests(image, image_name, params_harris, params_sift, params_star, thresholds, filter=False, n_pontos=['25', '50', '100']):
    if filter:
        algol_combinations = [('harris', 'sift_filter'), ('harris', 'star_filter'), ('sift_filter', 'star_filter')]
    else:    
        algol_combinations = [('harris', 'sift'), ('harris', 'star'), ('sift', 'star')]

    df = pd.DataFrame(columns=['combination', 'qtd_kp1', 'qtd_kp2', 'qtd_matches_kp1',
                                'qtd_matches_kp2', 'precision_1', 'precision_2'])
    
    for comb in algol_combinations:
        for i,n in enumerate(n_pontos):
            image_ = image.copy()
            params1 = return_params_combination(comb[0], params_harris, params_sift, params_star, i)
            params2 = return_params_combination(comb[1], params_harris, params_sift, params_star, i)
            
            result = compare_detectors_keypoints(image_, comb[0], comb[1], params1, params2, thresholds[i])
            
            kp_image, matches_kp1, matches_kp2, kp1, kp2 = result


            df.loc[len(df.index)] = [comb, len(kp1), len(kp2), len(matches_kp1), len(matches_kp2), len(matches_kp1) / len(kp1),
                               len(matches_kp2) / len(kp2) ]

            path_img = f'../results/first_part_detectors/{image_name}_matches_{comb[0]}_{comb[1]}_{n}_pontos.jpg'
            cv2.imwrite(path_img, kp_image)

    return df

### dsc07631.jpg

In [478]:
params_harris = {
    'threshold' : [0.27152, 0.078, 0.053],
    'blockSize' : [2, 5, 3],
    'ksize' : [3, 5, 9],
    'k' : [0.04, 0.04, 0.06]
}

params_sift = {
    'nfeatures': [34, 69, 143],
    'nOctaveLayers' : [3, 10, 10],
    'contrastThreshold' : [0.15, 0.05, 0.01],
    'edgeThreshold' : [2, 10, 3],
    'sigma' : [1.6, 3.2, 3.2],
    'enable_precise_upscale': [True, True, True]
}

params_star = {
    'max_size': [10, 16, 10],
    'response_threshold': [70, 55, 30],
    'line_threshold_projected': [10, 5, 5],
    'line_threshold_binarized': [10, 5, 5],
    'suppress_nonmax_size': [10, 16, 10]
}

thresholds = [5,5,5]

In [479]:
image = cv2.imread('../data/imgs/dsc07631.jpg')
# img = cv2.cvtColor(image, cv2.COLOR_RGB2Lab)
# clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8,8))
# img[:,:,0] = clahe.apply(img[:,:,0])
# img = cv2.cvtColor(img, cv2.COLOR_Lab2RGB)

image = cv2.resize(image, (640,480))

In [480]:
run_all_tests(image, 'dsc07631', params_harris, params_sift, params_star, thresholds)

,combination,qtd_kp1,qtd_kp2,qtd_matches_kp1,qtd_matches_kp2,precision_1,precision_2
0,"(harris, sift)",25,25,8,8,0.32,0.320000
1,"(harris, sift)",50,50,13,13,0.26,0.260000
2,"(harris, sift)",100,100,22,22,0.22,0.220000
3,"(harris, star)",25,26,5,5,0.20,0.192308
4,"(harris, star)",50,51,15,15,0.30,0.294118
5,"(harris, star)",100,100,26,26,0.26,0.260000
6,"(sift, star)",25,26,10,10,0.40,0.384615
7,"(sift, star)",50,51,23,23,0.46,0.450980
8,"(sift, star)",100,100,36,36,0.36,0.360000


### dsc02651.jpg

In [543]:
params_harris = {
    'threshold' : [0.43, 0.33, 0.14],
    'blockSize' : [3, 6, 6],
    'ksize' : [3, 5, 5],
    'k' : [0.04, 0.04, 0.04]
}

params_sift = {
    'nfeatures': [37, 60, 125],
    'nOctaveLayers' : [3, 10, 10],
    'contrastThreshold' : [0.15, 0.05, 0.01],
    'edgeThreshold' : [2, 10, 3],
    'sigma' : [1.6, 3.2, 3.2],
    'enable_precise_upscale': [True, True, True]
}

params_star = {
    'max_size': [11, 16, 16],
    'response_threshold': [71, 64, 48],
    'line_threshold_projected': [10, 6, 5],
    'line_threshold_binarized': [5, 6, 5],
    'suppress_nonmax_size': [8, 16, 8]
}

thresholds = [5,5,5]

In [544]:
# image = cv2.imread('../data/imgs/dsc07631.jpg')
image = cv2.imread('../data/imgs/dsc02651.jpg')
# img = cv2.cvtColor(image, cv2.COLOR_RGB2Lab)
# clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8,8))
# img[:,:,0] = clahe.apply(img[:,:,0])
# img = cv2.cvtColor(img, cv2.COLOR_Lab2RGB)

image = cv2.resize(image, (640,480))

In [545]:
run_all_tests(image, 'dsc02651', params_harris, params_sift, params_star, thresholds)

,combination,qtd_kp1,qtd_kp2,qtd_matches_kp1,qtd_matches_kp2,precision_1,precision_2
0,"(harris, sift)",25,25,5,5,0.20,0.20
1,"(harris, sift)",50,50,18,18,0.36,0.36
2,"(harris, sift)",100,100,31,31,0.31,0.31
3,"(harris, star)",25,25,4,4,0.16,0.16
4,"(harris, star)",50,50,23,23,0.46,0.46
5,"(harris, star)",100,100,33,33,0.33,0.33
6,"(sift, star)",25,25,11,11,0.44,0.44
7,"(sift, star)",50,50,30,30,0.60,0.60
8,"(sift, star)",100,100,63,63,0.63,0.63


### IMG-20240114-WA0043.jpg

In [1045]:
params_harris = {
    'threshold' : [0.005],
    'blockSize' : [6],
    'ksize' : [3],
    'k' : [0.04]
}

params_sift = {
    'nfeatures': [55],
    'nOctaveLayers' : [1],
    'contrastThreshold' : [0.02],
    'edgeThreshold' : [5],
    'sigma' : [1.6],
    'enable_precise_upscale': [True]
}

params_star = {
    'max_size': [10],
    'response_threshold': [10],
    'line_threshold_projected': [5],
    'line_threshold_binarized': [5],
    'suppress_nonmax_size': [1]
}

thresholds = [5]

In [1046]:
# image = cv2.imread('../data/imgs/dsc07631.jpg')
# image = cv2.imread('../data/imgs/dsc02651.jpg')
image = cv2.imread('../data/imgs_teste_3/IMG-20240114-WA0043.jpg')

# img = cv2.cvtColor(image, cv2.COLOR_RGB2Lab)
# clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8,8))
# img[:,:,0] = clahe.apply(img[:,:,0])
# img = cv2.cvtColor(img, cv2.COLOR_Lab2RGB)

image = cv2.resize(image, (640,480))

In [1047]:
run_all_tests(image, 'IMG-20240114-WA0043.jpg', params_harris, params_sift, params_star, thresholds, filter=True, n_pontos=['20'])

,combination,qtd_kp1,qtd_kp2,qtd_matches_kp1,qtd_matches_kp2,precision_1,precision_2
0,"(harris, sift_filter)",22,25,13,13,0.590909,0.520000
1,"(harris, star_filter)",22,21,20,20,0.909091,0.952381
2,"(sift_filter, star_filter)",25,21,12,12,0.480000,0.571429
